# Rename your subject folders  
  
Here we are going to change our subject directory names to match the `sub-X` format for BIDS processing, where `X` can be any identifier of our choice.

In [1]:
# load in packages
import os, glob, shutil
from datetime import datetime 
import re

First step is to get the path to the folders. We are simply renaming here and not moving anything, therefore we only need a single path.

In [ ]:
# first step is to get the path to our raw dicoms subject folders 
MAIN_PATH = "/projects/niblab/bids_projects/raw_data/continuing_studies/BRO"
# now we have the path, we gather the subject folders and hold them in a list 
# to do this we need to match our folders with a keyword to grab the correct them,
# in this example we use the '*' because every folder in the path is relevant. 
# this may be unique for your data and can be tricky if they have varying names
# the goal is to try to find a common keyword among them all that can be matched 
SUBJECT_FOLDERS=sorted(glob.glob(os.path.join(MAIN_PATH, "*")))
# now to verify we have grabbed the right folders we simply print the list to view it
print(SUBJECT_FOLDERS)

Great now we have our folders, next step is to rename them! This is where it becomes difficult to generalize the process because every initial name is different and the target name might be different, and we might have multiple sessions. So below I will show multiple examples that you can modify for your purpose.

To start I am going to show how to organize multiple sessions:

In [ ]:
# If we have multiple sessions, we are first going to create directories to hold the 
# individual sessions then we will rename them once they are in their corresponding session directory.
# Determing the sessions can be as simple as splitting the directories based on the identifier in the name,
# such as "v1" vs. "v2", or we may have to something more complex such as comparing dates, to identify which is which. 

# step 1: make your directories -- you will make as many as needed 
# in this example we make 2 directories for 2 sessions, and because we will use the directory paths in the future, 
# we will go ahead and save the paths to variables
SES1_PATH = os.path.join(MAIN_PATH, "ses-1")
SES2_PATH = os.path.join(MAIN_PATH, "ses-2")
#check our paths before making
print(SES1_PATH + "\n" + SES2_PATH)
#create directories
os.makedirs(SES1_PATH)
os.makedirs(SES2_PATH)

# Now we will go about identifying the sessions and move them to their corresponding directories

# so the example data we are using is unique, some names have a direct session identifier and others have dates we have to
# sort. So below we are separting the two into sub lists. 
# To seperate them, we identify unique keywords that we can divide them with. For instance this may be a common date like "2017",
# or it may be an identifer such as "V1" and "V2". 
# Here we were able to see that the folders with session identifiers all have "Bro_" in them, and the others do not, therefore
# we can confidently seperate based on that. However we always check our lists once we have divided them to ensure we didn't mess up!
sess_id_folders = []
date_id_folders = []
# we must start with looping through the list to check each folder
for sub_dir in SUBJECT_FOLDERS:
    if "Bro_" in sub_dir:
        sess_id_folders.append(sub_dir)
    else:
        date_id_folders.append(sub_dir)
        
        
# now that we have seperated our folders we can now move them into the correct folder!


# Case A: grabbing session identifier in the name -- this is when we have a clear identifier in the name
# since the identifier is in the name, this process is as simple as checking the ID and moving it into the correct folder
# Here "V2" corresponds to session 1 and "V3" corresponds to session 2
for sub_dir in sess_id_folders:
    # grab the folder name
    name = sub_dir.split("/")[-1]
    # check identifier and move into folder
    # !note -- I always like to ensure I am identifying correctly before moving anything to avoid complications therefore 
    # check the identification and name match to what you want by simplying printing the results as seen below!
    if "V2" in name:
        print("V2 ",name)
        print("moving %s >>----> %s"%(sub_dir, SES1_PATH))
        shutil.move(sub_dir, SES1_PATH)
    elif "V3" in name:
        print("V3 ",name)
        print("moving %s >>----> %s"%(sub_dir, SES2_PATH))
        shutil.move(sub_dir, SES2_PATH)
    else:
        pass
    
    
# Case B: date comparison 
# in this case we don't have a clear identification but need to compare dates 
# this can be tricky and unique, therefore I hope the process below can help guide you
date_dict = {}
for sub_dir in sorted(date_id_folders):
    name = sub_dir.split("/")[-1]
    date = name.split("_")[-1]
    sub = name.split("_")[1]
    if sub not in date_dict:
        date_dict[sub] = []
    date_dict[sub].append(date)
    
for sub in date_dict:
    # if there are not two dates that means we do not have the other sesssion and thus can not compare and for now
    # we will pass
    if len(date_dict[sub]) < 2:
        pass
    else:
        new_list = []
        for date in date_dict[sub]:
            x = re.split('(\d+)', date)
            x.remove('')
            string=",".join(x)
            string = string.rstrip(",")
            new_date = datetime.strptime(string, '%d,%b,%Y')
            new_list.append(new_date)
        date_dict[sub] = new_list
for sub in date_dict:
    if len(date_dict[sub]) < 2:
        pass
    else:
        dateA = date_dict[sub][0] 
        dateB = date_dict[sub][1]
        dateA.strftime('%d/%b/%Y'))
        new_list = 
        
        

Now we have an organized directory we can rename! 

In [ ]:
# IF there were multiple sessions are input directories will be different
# than the main path

### for multi-sess:
SES1_SUBS = sorted(glob.glob(os.path.join(SES1_PATH, "Bbx*")))
SES2_SUBS = sorted(glob.glob(os.path.join(SES2_PATH, "Bbx*")))

for sub_dir in SES1_SUBS:
    name = sub_dir.split("/")[-1]
    _id = name.split("_")[1].split("W")[0]
    new_name = "sub-"+_id
    new_path = os.path.join(SES1_PATH, new_name)
    os.rename(sub_dir, new_path)
    
for sub_dir in SES2_SUBS:
    name = sub_dir.split("/")[-1]
    _id = name.split("_")[1].split("W")[0]
    new_name = "sub-"+_id
    new_path = os.path.join(SES2_PATH, new_name)
    print(new_path)
    os.rename(sub_dir, new_path)